# Part 1 Executive Summary – Cloud Service Usage

In [ ]:
import base64
import os, sys, json
import requests
import time, datetime
import base64
import pandas as pd
from IPython.core import display as ICD
import pprint as pp
import mysql.connector
from mysql.connector import errorcode
from dotenv import load_dotenv
load_dotenv()

In [ ]:
BASE_URL = os.getenv("MYSHYN_URL")

TENANT_ID = os.getenv("MYSHYN_TENANT_ID")

USERNAME = os.getenv("MYSHYN_USERNAME")
PASSWORD = os.getenv("MYSHYN_PASSWORD")

REGISTRY_API_KEY = os.getenv("MYSHYN_REGISTRY_API")

PROXIES = {
    "https" : os.getenv("PROXIES_HTTPS")
}

In [ ]:
HEADER = {
    'Content-Type': 'application/json'
}

In [ ]:
def generateDateAndLast7day(date: None):
    # Get date
    selectedTime = datetime.datetime.strptime(date, '%d/%m/%y')
    last7dayTime = selectedTime - datetime.timedelta(days=6)
    
    # Convert date to timestamp
    selectedTime = selectedTime.strftime('%Y-%m-%dT23:59:00')
    last7dayTime = last7dayTime.strftime('%Y-%m-%dT00:00:00')

    # Convert timestamp to date
    # nowTime = datetime.datetime.fromtimestamp(nowTime)
    # last7dayTime = datetime.datetime.fromtimestamp(last7dayTime)
    
    return last7dayTime, selectedTime

In [ ]:
last7dayTime, selectedTime = generateDateAndLast7day(date='07/02/23')
last7dayTime, selectedTime

In [ ]:
def getServiceCountStat(tenantId: None, selectedColumn: {}, tag: [], deviceHost: None, startDate: None, endDate: None):
    getServiceCountStatUrl = BASE_URL + '/shnapi/rest/reporting/csv/queryPerServiceCounts'
    print("URL  : " + getServiceCountStatUrl)
        
    BODY = {
        "tableVisibilityMap": selectedColumn,
        "eventFilter": {
            "tagIdentifiers": tag,
            "from":startDate,
            "to":endDate
        }
    }
    
    if deviceHost and deviceHost != []:
        BODY["eventFilter"]["customAttrDtos"] = [
            { "attributeColName":"custom1", "values": deviceHost }
        ] 
    
#     pp.pprint(BODY)
    
    r = requests.post(getServiceCountStatUrl, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), proxies=PROXIES, verify=False)
    
    if r.status_code == 200:
        return r.text
    else:
        print("[" + str(r.status_code) + "] Error : " + str(r.text))
        return None

In [ ]:
def generateSeletedColumns():
    return {
#         "Service Name" : True,
#         "Category" : True,
#         "Risk"  :True,
#         "Access Count" : True,
#         "Outbound" : True,
#         "Inbound" : True,
#         "Upload Count" : True,
#         "Upload Data" : True,
#         "Users" : True,
#         "Allowed" : True,
#         "Denied" : True
    }

In [ ]:
def extractDataToDataframe(data: None):
    res = [res.split(",") for res in data.split('\n')]
    res_df = pd.DataFrame(data=res[1:len(res)-1], columns=res[0]) #.dropna()
    
    del res
    
    columnToInt = ['Risk', 'Access Count', 'Outbound', 'Inbound', 'Upload Count', 'Upload Data', 'Users', 'Allowed', 'Denied', 'Allowed Data', 'Denied Data', 'Total Data']
    res_df[columnToInt] = res_df[columnToInt].apply(pd.to_numeric, errors='coerce')

    columnToDatetime = ['ServiceFirstUsed', 'ServiceLastUsed']
    res_df[columnToDatetime] = res_df[columnToDatetime].apply(pd.to_datetime, errors='coerce')
    
    return res_df

In [ ]:
def extractServiceByRisk(data: None):
    dataHigh_df = data[data["Risk"]>=7]
    dataMedium_df = data[(data["Risk"]>=4) & (data["Risk"]<7)]
    dataLow_df = data[(data["Risk"]>=0) & (data["Risk"]<4)]
    return dataHigh_df, dataMedium_df, dataLow_df

In [ ]:
def generateServiceStat(data: None, excludeService: None):
    CloudServices_df = extractDataToDataframe(data=data)
    
    # Exclude Allow Service
    CloudServices_df = CloudServices_df[~(CloudServices_df["Service Id"].isin(excludeService))]
    
    # Filter Allowed
    allowedServices_df = CloudServices_df[CloudServices_df["Allowed"]>0]
    
    # Filter Allowed with Risk
    allowwithHighRiskServices_df, allowwithMediumRiskServices_df, allowwithLowRiskServices_df = extractServiceByRisk(data=allowedServices_df)

    # Filter Allowed with Risk - CLoud Storage
    allowedCloudStorage_df = allowedServices_df[allowedServices_df["Category"]=="Cloud Storage"]
    allowwithHighRiskCloudStorage_df, allowwithMediumRiskCloudStorage_df, allowwithLowRiskCloudStorage_df = extractServiceByRisk(data=allowedCloudStorage_df)
    
    # Filter Denied
    deniedServices_df = CloudServices_df[CloudServices_df["Denied"]>0]

    return {
        "all" : CloudServices_df,
        "allowedServices" : {
            "all" : allowedServices_df,
            "highRisk" : allowwithHighRiskServices_df,
            "mediumRisk" : allowwithMediumRiskServices_df,
            "lowRisk" : allowwithLowRiskServices_df,
            "cloudStorage" : {
                "all" : allowedCloudStorage_df,
                "highRisk" : allowwithHighRiskCloudStorage_df,
                "mediumRisk" : allowwithMediumRiskCloudStorage_df,
                "lowRisk" : allowwithLowRiskCloudStorage_df,
            }
        },
        "deniedServices" : deniedServices_df
    }

In [ ]:
def printServiceStat(allData: None, allowedData: None, deniedData: None):
    print("\nTotal Service Statistic")                                                          
    print("All Cloud Service Discovered : %d services" % (len(allData)))
    print("All Cloud Service Allowed Traffic : %d services" % (len(allowedData["all"])))
    print("All Cloud Service Allowed Traffic by Risk : %d(%d,%d,%d) services" % (len(allowedData["all"]),
                                                                                 len(allowedData["highRisk"]),
                                                                                 len(allowedData["mediumRisk"]),
                                                                                 len(allowedData["lowRisk"])))
    print("Cloud Storage Services Allowed Traffic : %d(%d,%d,%d) services" % (len(allowedData["cloudStorage"]["all"]),
                                                                              len(allowedData["cloudStorage"]["highRisk"]),
                                                                              len(allowedData["cloudStorage"]["mediumRisk"]),
                                                                              len(allowedData["cloudStorage"]["lowRisk"])))
    print("All Cloud Service Denied Traffic : %d services" % (len(deniedData)))
    

In [ ]:
fp_tag = 0
pa_tag = 0
wah_tag = 0

AllowServiceCSPIDList = ["0"]

# Device Host // Custom 1 Tag
remote_type = ["A"]
oversea_type = ["A"]

selectedColumn = generateSeletedColumns()

In [ ]:
# Forcepoint Proxy
fpServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[fp_tag],
                                    deviceHost=None,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
fpServiceCounts = generateServiceStat(data=fpServiceCounts, excludeService=AllowServiceCSPIDList)

# Remote Access
rmServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[pa_tag],
                                    deviceHost=remote_type,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
rmServiceCounts = generateServiceStat(data=rmServiceCounts, excludeService=AllowServiceCSPIDList)

# Oversea Branch
ozServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[pa_tag],
                                    deviceHost=oversea_type,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
ozServiceCounts = generateServiceStat(data=ozServiceCounts, excludeService=AllowServiceCSPIDList)

# Work Anywhere, Skyhigh
wawServiceCounts = getServiceCountStat(tenantId=TENANT_ID,
                                    selectedColumn=selectedColumn,
                                    tag=[wah_tag],
                                    deviceHost=None,
                                    startDate=last7dayTime,
                                    endDate=selectedTime)
wawServiceCounts = generateServiceStat(data=wawServiceCounts, excludeService=AllowServiceCSPIDList)


In [ ]:
print("\nPart 1 Executive Summary – Cloud Service Usage : ")

print("\nHQ : ")
printServiceStat(allData=fpServiceCounts["all"],
                 allowedData=fpServiceCounts["allowedServices"],
                 deniedData=fpServiceCounts["deniedServices"])
ICD.display(fpServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nRemote Access : ")
printServiceStat(allData=rmServiceCounts["all"],
                 allowedData=rmServiceCounts["allowedServices"],
                 deniedData=rmServiceCounts["deniedServices"])
ICD.display(rmServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nOversea Branch: ")
printServiceStat(allData=ozServiceCounts["all"],
                 allowedData=ozServiceCounts["allowedServices"],
                 deniedData=ozServiceCounts["deniedServices"])
ICD.display(ozServiceCounts["allowedServices"]["highRisk"].head(5))

print("\nWork Anywhere Gateway : ")
printServiceStat(allData=wawServiceCounts["all"],
                 allowedData=wawServiceCounts["allowedServices"],
                 deniedData=wawServiceCounts["deniedServices"])
ICD.display(wawServiceCounts["allowedServices"]["highRisk"].head(5))

# Part 2 Anomalies Activities UEBA

Go to portal and Capture Image

In [ ]:
last7dayTime, selectedTime

In [ ]:
# Anomalies
# Status = new, Service Name = Exchange, Onedrive, Sharepoint
# Capture Image

{
    "type":"in_search_query",
    "field":"incident_detail.services.name",
    "values":["sharepoint","onedrive","exchange online"]
},
{
  "type": "equal_search_query",
  "value": "new",
  "field": "workflow.status"
}
{
    "type":"equal_search_query",
    "value":"2",
    "field":"severity"
}

In [ ]:
# hqAllCloudServices_df[["Service Id", "Service Name"]][hqAllCloudServices_df["Service Name"].str.contains("microsoft", case=False)]

In [ ]:
# eventFilter = {
# #     "serviceidentifiers": ["2799", "3210"],
#     "from":last7dayTime,
#     "to":nowTime,
# }

In [ ]:
# BODY = {
#     "templateId" : 10,
#     "eventFilter" : eventFilter
# }

# BODY

In [ ]:
# url = BASE_URL + '/shnapi/rest/reporting/csv/queryAnomaliesForUser'
# print("URL  : " + url)
# r = requests.post(url, headers=HEADER, params={}, data=json.dumps(BODY).encode("utf-8"), auth=(USERNAME,PASSWORD), verify=False)
# r.status_code

In [ ]:
# res = [res.split(",") for res in r.text.split('\n')]
# res_df = pd.DataFrame(data=res[1:], columns=res[0])#.dropna()
# res_df

# Part 3 - Activities

In [ ]:
def generateDateAndLast7day(startdate: None, enddate: None) -> object:
    # Get date
    startDate = datetime.datetime.strptime(startdate, '%d/%m/%y')
    endDate = datetime.datetime.strptime(enddate, '%d/%m/%y')

    # Convert date to timestamp
    startDate = startDate.strftime('%Y-%m-%dT00:00:00.000+07:00')
    endDate = endDate.strftime('%Y-%m-%dT23:59:00.000+07:00')

    return startDate, endDate

In [ ]:
def getActivityStat(XACCESSTOKEN: str, SERVICES: list, startdate: None, enddate: None) -> list:
    try:
        getActivityStatURL = BASE_URL + '/neo/ta-activity-service/v1/tp/activity/facet'

        headers = {
            'Accept': 'application/json, text/plain, */*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/json;charset=UTF-8',
            'x-access-token': str(XACCESSTOKEN)
        }

        payload = {
            "search_query": {
                "type": "and_search_query",
                "queries": [
                    {
                        "type": "in_search_query",
                        "field": "serviceName",
                        "values": SERVICES
                    },
                    {
                        "field": "created_on_date",
                        "lower_bound": startdate,
                        "type": "between_search_query",
                        "upper_bound": enddate
                    }
                ]
            },
            "paging_identifiers": {},
            "limit": 300,
            "is_descending": "false",
            "timezone": "Asia/Bangkok",
            "selected_facets": [
                "Service Name",
                "Instance"
            ]
        }

        response = requests.request("POST", getActivityStatURL, params={}, headers=headers, data=json.dumps(payload).encode("utf-8"), proxies=PROXIES, verify=False)
        if response.status_code == 200:
            return response.json()
        else:
            response = response.json()
            print("[Error] Request Error.")
            pp.pprint(response)
            sys.exit()
    except:
        print("[Error] Connection Error.")
        sys.exit()

In [ ]:
def checkActivityType(serviceGroup: str, activityType: str, data: list) -> dict:
    for d in data:
        if d['label'] == activityType:
            return {
                "service": serviceGroup,
                'label': d['label'],
                'name': d['name'],
                "count" : d['count']
            }

    return {
        "service": serviceGroup,
        'label': activityType,
        'name': "N/A_NOTFOUNDONTHISPERIOD",
        "count" : "0"
    }

In [ ]:
def extractActivityStats(jsondata: list) -> list: 
    for d in jsondata:
        if d["label"] == "TOP_ACTIVITY_TYPE":
            print("%s, Total Activities : %d" %(d["name"], d["count"]))
            return d["child_facets"] if "child_facets" in d else None
    return None

In [ ]:
def extractAllActivityType(serviceGroup: str, allActivityType: list, data: list) -> list:
    activityTypeResults = []
    for activityType in allActivityType:
        activityTypeResults.append(checkActivityType(serviceGroup=serviceGroup, activityType=activityType, data=data))
    return activityTypeResults


In [ ]:
SERVICES = {
    "ONEDRIVE_SHAREPOINT": {
        "query": ["SharePoint","OneDrive"],
        "activitytype": [
            "FILE_ACCESSED",
            "FILEACCESSEDEXTENDED",
            "FILEPREVIEWED",
            "FILE_MODIFIED",
            "FILE_UPLOADED",
            "FILEMODIFIEDEXTENDED",
            "ADDED_TO_GROUP",
            "FOLDERCREATED",
            "FILE_DELETED",
            "FILERECYCLED",
            "FILE_OR_FOLDER_SHARED",
            "ADDEDTOSECURELINK",
            "FILE_DOWNLOADED",
            "FILE_RENAMED",
            "FILE_MOVED",
            "FILES_DOWNLOADED_TO_COMPUTER",
            "FILES_UPLOADED_TO_DOCUMENT_LIBRARY"
        ]
    },
    "EXCHANGE" : {
        "query": ["Exchange Online"],
        "activitytype": [
            "CREATE_ITEM",
            "UPDATE_ITEM",
            "MAILITEMSACCESSED",
            "MODIFYFOLDERPERMISSIONS",
            "SOFT_DELETE",
            "HARD_DELETE",
            "MAILBOX_LOGIN",
            "SEND_AS",
            "SEND_ON_BEHALF",
            "SEND",
            "MOVE_TO_DELETED_ITEMS",
            "NEW-FOLDERMOVEREQUEST",
            "ADD-DISTRIBUTIONGROUPMEMBER"
        ]
    },
    "MSTEAM" : {
        "query": ["Microsoft - Teams"],
        "activitytype": [
            "MESSAGECREATEDNOTIFICATION",
            "MESSAGEUPDATEDNOTIFICATION",
            "MESSAGECREATEDHASLINK",
            "MESSAGEEDITEDHASLINK",
            "MEETINGPARTICIPANTDETAIL",
            "MESSAGEUPDATED",
            "MEETINGDETAIL",
            "MEMBERREMOVED",
            "TEAMSSESSIONSTARTED",
            "MESSAGEDELETED",
            "MEMBERADDED",
            "APPINSTALLED",
            "MEMBERROLECHANGED",
            "VIEWEDAPPROVALREQUEST"
        ]
    }
}

In [ ]:
serviceGroup, serviceGroupData = "ONEDRIVE_SHAREPOINT", SERVICES["ONEDRIVE_SHAREPOINT"]

startdate = "01/03/23"
enddate = "07/03/23"

In [ ]:
XACCESSTOKEN = ''

In [ ]:
startDate, endDate = generateDateAndLast7day(startdate=startdate, enddate=enddate)   

activityStat_data = getActivityStat(XACCESSTOKEN=XACCESSTOKEN, SERVICES=serviceGroupData["query"], startdate=startDate, enddate=endDate)
activityStat_data = extractActivityStats(jsondata=activityStat_data)
activityStat_data = extractAllActivityType(serviceGroup=serviceGroup, allActivityType=serviceGroupData["activitytype"], data=activityStat_data)

In [ ]:
activityStat_data = pd.DataFrame(activityStat_data)
activityStat_data

### Sensitive Database Zone
Remove Credential All Time

In [ ]:
host = ""
username = ""
password = "" 
database = ""

In [ ]:
def connectDB() -> object:
    try:
        dataBase = mysql.connector.connect(
            host = host,
            user = base64.b64decode(username).decode("utf-8", "ignore"),
            passwd = base64.b64decode(password).decode("utf-8", "ignore"),
            database = database,
            port = '3306',
            ssl_disabled = True
        )
        if dataBase.is_connected():
            return dataBase
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    sys.exit()

In [ ]:
def generatePeriodDate(startdate: None, enddate: None) -> object:
    # Get date
    startDate = datetime.datetime.strptime(startdate, '%d/%m/%y')
    endDate = datetime.datetime.strptime(enddate, '%d/%m/%y')

    # # Convert date to timestamp
    startDate = startDate.strftime('%y%m%d')
    endDate = endDate.strftime('%y%m%d')

    return f"{startDate}TO{endDate}"

In [ ]:
def appendCASBActivityToDB(XACCESSTOKEN: str, startdate: None, enddate: None) -> None:
    startDate, endDate = generateDateAndLast7day(startdate=startdate, enddate=enddate)   

    activityStat_data = getActivityStat(XACCESSTOKEN=XACCESSTOKEN, SERVICES=serviceGroupData["query"], startdate=startDate, enddate=endDate)
    activityStat_data = extractActivityStats(jsondata=activityStat_data)
    activityStat_data = extractAllActivityType(serviceGroup=serviceGroup, allActivityType=serviceGroupData["activitytype"], data=activityStat_data)
    
    periodDate = generatePeriodDate(startdate=startdate, enddate=enddate)

    dataBase = self.connectDB()
    dataBaseCursor = dataBase.cursor()

    add_activity = ("INSERT INTO user_activity(service, label, name, count, period)"
                   "VALUES (%s, %s, %s, %s, %s)")

    activitiesDatasforInsert = [(s["service"], s["label"], s["name"], s["count"], periodDate) for s in serviceGroupsAllActivityStat]
    # print(activitiesDatasforInsert)

    dataBaseCursor.executemany(add_activity, activitiesDatasforInsert)
    dataBase.commit()

    dataBaseCursor.close()
    dataBase.close()

In [ ]:
def getActivityDBbySelectedPeriod(selectedPeriod: []):
    dataBase = connectDB()
    dataBaseCursor = dataBase.cursor()

    format_strings = ','.join(['%s'] * len(selectedPeriod))

    query = ("SELECT * FROM user_activity "
            "WHERE period IN (%s)" % format_strings)

    dataBaseCursor.execute(query, tuple(selectedPeriod))

    data = [a for a in dataBaseCursor]

    dataBaseCursor.close()
    dataBase.close()
    
    return data

In [ ]:
def generateSelected5PeriodDate(startdate: None, enddate: None) -> object:
    # Get date
    startDate = datetime.datetime.strptime(startdate, '%d/%m/%y')
    endDate = datetime.datetime.strptime(enddate, '%d/%m/%y')

    # Convert date to timestamp
    startDate_str = startDate.strftime('%y%m%d')
    endDate_str = endDate.strftime('%y%m%d')

    selectedPeriod = [f"{startDate_str}TO{endDate_str}"]
    
    for i in range(1, 5):
        startDate = startDate - datetime.timedelta(days=7)
        endDate = endDate - datetime.timedelta(days=7)
        
         # Convert date to timestamp
        startDate_str = startDate.strftime('%y%m%d')
        endDate_str = endDate.strftime('%y%m%d')

        selectedPeriod.append(f"{startDate_str}TO{endDate_str}")
    return selectedPeriod

In [ ]:
selectedPeriod = generateSelected5PeriodDate(startdate=startdate, enddate=enddate)
selectedPeriod = selectedPeriod[::-1]
selectedPeriod

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
activities = getActivityDBbySelectedPeriod(selectedPeriod=selectedPeriod)
activities = pd.DataFrame(activities, columns =['id', "service", "label", "name", "count", "period"])
activities["count"] = activities["count"].astype(int)
activities

In [ ]:
for serviceGroup, serviceGroupData in SERVICES.items():
    print(serviceGroup)
    
    payloadPlot = {}
    for activityType in serviceGroupData['activitytype']:
        selectdAcitivity = list(activities[(activities["service"]==serviceGroup) & (activities["label"]==activityType)]["count"])
        payloadPlot[activityType] = selectdAcitivity
    #     payloadPlot[activityType] = pd.DataFrame({activityType: selectdAcitivity}, index=selectedPeriod)

    dfForPlot = pd.DataFrame(payloadPlot, index=selectedPeriod)
    dfForPlot.plot.line(title=serviceGroup, subplots=True, grid=True, sharey=False, sharex=False, legend=True, layout=(5,4), figsize=(35,15))
    plt.savefig(f'{serviceGroup}.jpg')

In [ ]:
plt.get_fignums()